In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
# load the data set

Data = pd.read_excel(r"C:\Users\HP\OneDrive\Desktop\Assignment DS (Husen)\Assignment No 18\Forecasting\CocaCola_Sales_Rawdata.xlsx")
Data.head()

,Quarter,Sales
0,Q1_86,1734.827000
1,Q2_86,2244.960999
2,Q3_86,2533.804993
3,Q4_86,2154.962997
4,Q1_87,1547.818996


In [3]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Quarter  42 non-null     object 
 1   Sales    42 non-null     float64
dtypes: float64(1), object(1)
memory usage: 800.0+ bytes


In [4]:
Data.shape

(42, 2)

In [5]:
# copy the orignal data

df = Data.copy()

In [6]:
df.head(10)

,Quarter,Sales
0,Q1_86,1734.827000
1,Q2_86,2244.960999
2,Q3_86,2533.804993
3,Q4_86,2154.962997
4,Q1_87,1547.818996
5,Q2_87,2104.411995
6,Q3_87,2014.362999
7,Q4_87,1991.746998
8,Q1_88,1869.049999
9,Q2_88,2313.631996


In [7]:
df= df.set_index('Quarter')

In [8]:
df.head()

,Sales
Quarter,
Q1_86,1734.827000
Q2_86,2244.960999
Q3_86,2533.804993
Q4_86,2154.962997
Q1_87,1547.818996


In [9]:
train = pd.read_excel(r"C:\Users\HP\OneDrive\Desktop\Assignment DS (Husen)\Assignment No 18\Forecasting\CocaCola_Sales_Rawdata.xlsx", header=0, index_col=0, parse_dates=True)

In [10]:
X = train.values
X = X.astype('float32')
train_size = int(len(X) * 0.50)
train, test = X[0:train_size], X[train_size:]

In [11]:
train

array([[1734.827],
       [2244.961],
       [2533.805],
       [2154.963],
       [1547.819],
       [2104.412],
       [2014.363],
       [1991.747],
       [1869.05 ],
       [2313.632],
       [2128.32 ],
       [2026.829],
       [1910.604],
       [2331.165],
       [2206.55 ],
       [2173.968],
       [2148.278],
       [2739.308],
       [2792.754],
       [2556.01 ],
       [2480.974]], dtype=float32)

In [12]:
history = [x for x in train]
predictions = list()
for i in range(len(test)):
    yhat = history[-1]
    predictions.append(yhat)
# observation
    obs = test[i]
    history.append(obs)
    print('>Predicted=%.3f, Expected=%.3f' % (yhat, obs))
# performance
rmse = sqrt(mean_squared_error(test, predictions))
print('RMSE: %.3f' % rmse)

>Predicted=2480.974, Expected=3039.523
>Predicted=3039.523, Expected=3172.116
>Predicted=3172.116, Expected=2879.001
>Predicted=2879.001, Expected=2772.000
>Predicted=2772.000, Expected=3550.000
>Predicted=3550.000, Expected=3508.000
>Predicted=3508.000, Expected=3243.860
>Predicted=3243.860, Expected=3056.000
>Predicted=3056.000, Expected=3899.000
>Predicted=3899.000, Expected=3629.000
>Predicted=3629.000, Expected=3373.000
>Predicted=3373.000, Expected=3352.000
>Predicted=3352.000, Expected=4342.000
>Predicted=4342.000, Expected=4461.000
>Predicted=4461.000, Expected=4017.000
>Predicted=4017.000, Expected=3854.000
>Predicted=3854.000, Expected=4936.000
>Predicted=4936.000, Expected=4895.000
>Predicted=4895.000, Expected=4333.000
>Predicted=4333.000, Expected=4194.000
>Predicted=4194.000, Expected=5253.000
RMSE: 527.148


In [13]:
# evaluate an ARIMA model for a given order (p,d,q) and return RMSE
def evaluate_arima_model(X, arima_order):
# prepare training dataset
    X = X.astype('float32')
    train_size = int(len(X) * 0.50)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]

    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
# sample error
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse

In [14]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float('inf'), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(train, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except:
                    continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

In [15]:
train = pd.read_excel(r"C:\Users\HP\OneDrive\Desktop\Assignment DS (Husen)\Assignment No 18\Forecasting\CocaCola_Sales_Rawdata.xlsx", header=0, index_col=0, parse_dates=True)
X = train.values
X = X.astype('float32')

In [16]:
model = ARIMA(X, order=(3,1,0))
model_fit = model.fit()
forecast=model_fit.forecast(steps=10)[0]
model_fit.predict(1, 59)

array([1705.11569575, 1995.62758005, 2054.43326532, 1881.70859987,
       1969.02167961, 2462.27104787, 1942.04497247, 1803.54503108,
       1985.54446835, 2204.86823835, 1951.89760745, 1997.52692028,
       2126.24763659, 2266.36603771, 2016.0011251 , 2079.91241996,
       2242.94026494, 2496.57237103, 2352.6798839 , 2348.02227774,
       2661.90201164, 2944.46094677, 2741.85514894, 2655.59710326,
       2971.11952263, 3402.01068292, 3016.01737104, 3029.05032316,
       3354.81316177, 3765.28601704, 3235.1300461 , 3288.37211895,
       3674.89224029, 4014.97804105, 3699.60991952, 3669.52097733,
       4194.8535052 , 4757.67552237, 4209.35146014, 4112.8137597 ,
       4684.76893665, 5124.04146552, 4484.82705919, 4375.28237608,
       4949.77499647, 5063.00572374, 4646.64722844, 4487.38984335,
       4809.24536723, 4971.05241999, 4738.13622186, 4577.64423737,
       4744.44582763, 4892.77906702, 4778.83849935, 4645.88710984,
       4720.17885791, 4835.5154866 , 4790.83956454])